## Scraping IMO Board

In [1]:
# Import library needed
import os
import json
import pickle
import ctypes
import pygame
import datetime
import requests
import webbrowser
import numpy as np
import pandas as pd

from time import sleep
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.support import expected_conditions as EC

from bs4 import BeautifulSoup

from selenium.webdriver.firefox.options import Options
root_url = "https://www.imo-official.org/problems.aspx"
opts = Options().add_argument("--private")

# Configuration
os.environ['PATH'] += r";C:/SeleniumDrivers"
firefox_driver_path = "C:/SeleniumDrivers/geckodriver.exe"
driver = webdriver.Firefox(executable_path=firefox_driver_path, options=opts)
wait = WebDriverWait(driver, 20)
driver.maximize_window()
driver.get(root_url)
sleep(2)


pygame 2.5.0 (SDL 2.28.0, Python 3.9.12)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
# Initialize how the data stored
data = {
    'id_key': [],
    'year': [],
    'country': [],
    'all': [],
    'male': [],
    'female': [],
    'p1': [],
    'p2': [],
    'p3': [],
    'p4': [],
    'p5': [],
    'p6': [],
    'p7': [],
    'total': []
}


In [3]:
# The IMO start from 1959 - 2023
start, end = 1959, 2023
years = [str(i) for i in range(start, end+1)]
years.remove('1980')  # imo 1980 tidak ada
target_url = "https://www.imo-official.org/year_country_r.aspx?year="

print(f"example url → {target_url+years[0]}")

example url → https://www.imo-official.org/year_country_r.aspx?year=1959


In [4]:
# year > 1988 ada ketambahan award 'HM'
# 1960 & 1962 saja yang sampe P7

id_key = 1
for year in years:
    driver.get(target_url + year)
    
    # Wait for the table to be visible
    table = WebDriverWait(driver, 10).until(
        EC.visibility_of_element_located((By.TAG_NAME, 'table'))
    )

    soup = BeautifulSoup(driver.page_source, 'html.parser')
    table = soup.find('table')
    thead = soup.find('thead')

    # Find the <th> element with the title "Honourable mention" inside <thead>
    # Find this element specific = '<th title="Honourable mention">HM</th>'
    honourable_mention_th = thead.find('th', title='Honourable mention')
    
    # If '<th title="Honourable mention">HM</th>' exists
    if honourable_mention_th is not None:
        for row in table.find_all('tr'):
            cells = row.find_all('td')
            column = len(cells)

            if len(cells) != 0:
                country = str(cells[0].text.strip())
                all = (None if cells[1].text.strip() == "" else int(cells[1].text.strip()))
                male = (-1 if cells[2].text.strip() == "" else int(cells[2].text.strip()))
                female = (-1 if cells[3].text.strip() == "" else int(cells[3].text.strip()))

                p1 = (None if cells[4].text.strip() == "" else int(cells[4].text.strip()))
                p2 = (None if cells[5].text.strip() == "" else int(cells[5].text.strip()))
                p3 = (None if cells[6].text.strip() == "" else int(cells[6].text.strip()))
                p4 = (None if cells[7].text.strip() == "" else int(cells[7].text.strip()))
                p5 = (None if cells[8].text.strip() == "" else int(cells[8].text.strip()))
                p6 = (None if cells[9].text.strip() == "" else int(cells[9].text.strip()))
                p7 = None
                
                temp = cells[10].text.strip()
                total = (None if temp in [""] else int(cells[10].text.strip()))
                    
                # Store data
                data['id_key'].append(id_key)
                data['year'].append(int(year))
                
                data['country'].append(country)
                data['all'].append(all)
                data['male'].append(male)
                data['female'].append(female)
                
                data['p1'].append(p1)
                data['p2'].append(p2)
                data['p3'].append(p3)
                data['p4'].append(p4)
                data['p5'].append(p5)
                data['p6'].append(p6)
                data['p7'].append(p7)
                
                data['total'].append(total)
                id_key += 1
    
    else:
        for row in table.find_all('tr'):
            cells = row.find_all('td')
            column = len(cells)

            if len(cells) != 0:
                # P1-P6
                if column == 17:
                    country = str(cells[0].text.strip())
                    all = (None if cells[1].text.strip() == "" else int(cells[1].text.strip()))
                    male = (-1 if cells[2].text.strip() == "" else int(cells[2].text.strip()))
                    female = (-1 if cells[3].text.strip() == "" else int(cells[3].text.strip()))

                    p1 = (None if cells[4].text.strip() == "" else int(cells[4].text.strip()))
                    p2 = (None if cells[5].text.strip() == "" else int(cells[5].text.strip()))
                    p3 = (None if cells[6].text.strip() == "" else int(cells[6].text.strip()))
                    p4 = (None if cells[7].text.strip() == "" else int(cells[7].text.strip()))
                    p5 = (None if cells[8].text.strip() == "" else int(cells[8].text.strip()))
                    p6 = (None if cells[9].text.strip() == "" else int(cells[9].text.strip()))
                    p7 = None
                    
                    temp = cells[10].text.strip()
                    total = (None if temp in [""] else int(cells[10].text.strip()))
                    
                # P1-P7
                elif column == 18:
                    country = str(cells[0].text.strip())
                    all = (None if cells[1].text.strip() == "" else int(cells[1].text.strip()))
                    male = (-1 if cells[2].text.strip() == "" else int(cells[2].text.strip()))
                    female = (-1 if cells[3].text.strip() == "" else int(cells[3].text.strip()))

                    p1 = (None if cells[4].text.strip() == "" else int(cells[4].text.strip()))
                    p2 = (None if cells[5].text.strip() == "" else int(cells[5].text.strip()))
                    p3 = (None if cells[6].text.strip() == "" else int(cells[6].text.strip()))
                    p4 = (None if cells[7].text.strip() == "" else int(cells[7].text.strip()))
                    p5 = (None if cells[8].text.strip() == "" else int(cells[8].text.strip()))
                    p6 = (None if cells[9].text.strip() == "" else int(cells[9].text.strip()))
                    p7 = (None if cells[10].text.strip() == "" else int(cells[10].text.strip()))

                    temp = cells[11].text.strip()
                    total = (None if temp in [""] else int(cells[11].text.strip()))
                    
                # Store data
                data['id_key'].append(id_key)
                data['year'].append(int(year))
                
                data['country'].append(country)
                data['all'].append(all)
                data['male'].append(male)
                data['female'].append(female)
                
                data['p1'].append(p1)
                data['p2'].append(p2)
                data['p3'].append(p3)
                data['p4'].append(p4)
                data['p5'].append(p5)
                data['p6'].append(p6)
                data['p7'].append(p7)
                
                data['total'].append(total)
                id_key += 1

    # sleep(1)


# <div style="padding:20px;color:white;margin:0;font-size:30px;font-family:Georgia;text-align:left;display:fill;border-radius:5px;background-color:#7b6b59;overflow:hidden">SAVE to CSV</div>

In [6]:
# JANGAN DI JALANKAN LAGI.. YANG KOSONG SUDAH DI ISI MANUAL.. (MAKANYA JANGAN RUN INI!!) 
# TANPA tahun 1980

# df = pd.DataFrame(data)
# df.to_csv('imo_board.csv', index=False)

# display(df)

,id_key,year,country,all,male,female,p1,p2,p3,p4,p5,p6,p7,total
0,1,1959,Romania,8,6,-1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,249.0
1,2,1959,Hungary,8,8,-1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,233.0
2,3,1959,Czechoslovakia,8,5,-1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,192.0
3,4,1959,Bulgaria,8,6,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,131.0
4,5,1959,Poland,8,7,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,122.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3667,3668,2023,Uganda,5,4,1,4.0,0.0,0.0,2.0,1.0,0.0,NaN,7.0
3668,3669,2023,Cameroon,6,6,0,6.0,0.0,0.0,0.0,0.0,0.0,NaN,6.0
3669,3670,2023,Guatemala,2,0,2,4.0,0.0,0.0,1.0,0.0,0.0,NaN,5.0
3670,3671,2023,Kenya,6,6,0,4.0,0.0,0.0,0.0,0.0,0.0,NaN,4.0


In [6]:
# df.groupby('year')[['p1', 'p2', 'p3', 'p4', 'p5', 'p6', 'p7', 'total']].sum()